# Introduction

This notebook is the second in the Makemore series. In part 1 we implemented Makemore using a bigram model. We looked at ONE previous character and produced a probability distribution of what character is likely to come next. We did that using two different approaches and reached the same results. First, we used counts and normalized them into probabilities. Second, we used a simple neural network (i.e. linear layer) to produce those same probabilities.

The limit of our implementation in part 1 is that it only looks at ONE previous character; and because the bigram model took only ONE character as context, the prediction of the bigram were **not** good. If we were to use consider MORE than one character when predicting the next one, the approach we used in part-1 (storing everything in a tensor) becomes unsustainable because _the number of combinations we will have to store in a tensor grows exponentially with the amount characters we take as context_. For instance, if we were to take just three characters as context when making a prediction, we would have to store $27 \times 27 \times 27 = 19683$ possibilities in a tensor. That's way to many possibilities, the majority of these possibilities will have very few counts, Andrej said.

That's why for this second part, we are moving away from the Bigram model. This time, we are implementing a Multi-Layer Percepton to predict the next character. The modeling approach we are going to adopt follows the paper: [Bengio et al. 2003](https://www.jmlr.org/papers/volume3/bengio03a/bengio03a.pdf) 

# Bengio et al. 2003 (MLP language model) paper walkthrough

This paper, Andrej said, is not the first to have proposed the use of MLPs to predict the next character/token in a sequence, but was very influencial and is very often cited. Since the paper is long ($19$ pages) Andrej decided to give us gist of it, but invited us to read the entire work. This paper is what we are going to implement in this notebook.

# Re-building our training dataset

In [3]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt # For making figures
%matplotlib inline

In [4]:
# Read in all the words
words = open('names.txt', 'r').read().splitlines()
words[:8]

['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']

In [5]:
len(words)

32033

Now we build the vocabulary of characters and mapping from/to integers. It is important to recall that we are building a character-level language model, unlike the paper who is describing a word-level language model.

In [8]:
# Build the vocabulary of characters and mapping to/from integers
chars = sorted(list(set(''.join(words))))
stoi = {s:i + 1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
print(itos)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


We now want to set up the dataset such that we can feed training examples to the neural network easily. We are going to refurbish code we wrote in the previous tutorial. To form the dataset in the first part, we added the first part of the bigram (the context) in a tensor, `xs`... and the second part (the correct prediction) in another tensor, `ys`. 

We are doing something very similar here, but this time Andrej is adding MORE than ONE character as context. `X` will serve as the input to the neural network and `Y` will serve as the correct label.

In [10]:
# build the dataset

block_size = 3 # context length: how many characters do we take to predict the next one?
X, Y = [], []
for w in words[:5]:
  
  print(w)
  context = [0] * block_size
  for ch in w + '.':
    idx = stoi[ch]
    X.append(context)
    Y.append(idx)
    print(''.join(itos[i] for i in context), '--->', itos[idx])
    context = context[1:] + [idx] # crop and append
  
X = torch.tensor(X)
Y = torch.tensor(Y)

emma
... ---> e
..e ---> m
.em ---> m
emm ---> a
mma ---> .
olivia
... ---> o
..o ---> l
.ol ---> i
oli ---> v
liv ---> i
ivi ---> a
via ---> .
ava
... ---> a
..a ---> v
.av ---> a
ava ---> .
isabella
... ---> i
..i ---> s
.is ---> a
isa ---> b
sab ---> e
abe ---> l
bel ---> l
ell ---> a
lla ---> .
sophia
... ---> s
..s ---> o
.so ---> p
sop ---> h
oph ---> i
phi ---> a
hia ---> .
